<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/marco-canas/intro-Machine-Learning/blob/main/classes/class_18/class_18_preparar_datos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
  </td>
</table> 

# Clase 20 Continuación de la construcción del modelo de Machine Learning de regresión lineal

1. Plantear bien la pregunta.  

   * ¿Regresión o clasificación?
   * ¿Tipo de regresión y tipo de clasificación?

2. Exploración inicial.
   * Indicar la fuente de dónde se toman los datos.
   * Hacer explícita la función objetivo.
   * Decir cuáles son los atributos (descripción breve de cada uno)
   * Practicar una primera síntesis tabular y una exploración gráfica de los datos.

3. Preparar los datos para los algoritmos de aprendizaje.
   
   * Hacer separación inicial de datos para entrenar y para testear.
   * Explorar correlaciones lineales con la variable objetivo.
   * adicionar atributos que estén mejor correlacionados con la variable objetivo.
   * Llenar datos faltantes.
   * Codificar las variables categóricas. 
   * Estandarizar los datos.

In [10]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 

v = pd.read_csv('vivienda.csv')

from sklearn.model_selection import train_test_split

v_train, v_test = train_test_split(v, test_size = 0.2, random_state = 513) 

v = v_train

## Tratamiento de datos faltantes

Vimos anteriormente que el atributo `dormitorios` tiene algunos valores faltantes, así que arreglemos esto.

Tienes tres opciones:

1. Deshacerse de los distritos correspondientes.
2. Deshágase de todo el atributo.
3. Establezca los valores en algún valor (cero, la media, la mediana, etc.).

Puede lograr esto fácilmente usando los métodos `dropna()`, `drop()` y `fillna()`:

In [5]:
v.dropna(subset = ['dormitorios']) 

In [6]:
v.info() 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16348 entries, 18223 to 17044
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   longitud      16348 non-null  float64
 1   latitud       16348 non-null  float64
 2   antiguedad    16348 non-null  float64
 3   habitaciones  16348 non-null  float64
 4   dormitorios   16348 non-null  float64
 5   población     16348 non-null  float64
 6   hogares       16348 non-null  float64
 7   ingresos      16348 non-null  float64
 8   proximidad    16348 non-null  object 
 9   precio        16348 non-null  float64
dtypes: float64(9), object(1)
memory usage: 1.4+ MB


In [8]:
v.drop('dormitorios', axis = 1) 

In [11]:
median = v.dormitorios.median()

In [12]:
v['dormitorios'].fillna(median, inplace = True) 